In [7]:
# Cell 2: Import libraries
import pandas as pd
import numpy as np
import os
import pickle
from datetime import datetime

print("\nLibraries imported successfully")


print("=" * 60)
print("LOADING ALL VARIABLES FROM Q2")
print("=" * 60)

# Load the production dictionary from Q2
%store -r production_data

# Check if loaded successfully
if 'production_data' in dir() and production_data is not None:
    print(f"Loaded production_data dictionary with {len(production_data)} items")
    
    # Show everything in production_data
    print("\n" + "=" * 60)
    print("COMPLETE DATA LOADED FROM Q2")
    print("=" * 60)
    
    # Extract all variables
    daily_production_df = production_data.get('daily_production_df')
    avg_daily_kwh = production_data.get('avg_daily_kwh')
    max_daily_kwh = production_data.get('max_daily_kwh')
    min_daily_kwh = production_data.get('min_daily_kwh')
    best_day = production_data.get('best_day')
    best_day_value = production_data.get('best_day_value')
    worst_day = production_data.get('worst_day')
    worst_day_value = production_data.get('worst_day_value')
    
    february_total_kwh = production_data.get('february_total_kwh')
    february_days = production_data.get('february_days')
    
    hourly_avg_production = production_data.get('hourly_avg_production')
    peak_hour = production_data.get('peak_hour')
    peak_hour_value = production_data.get('peak_hour_value')
    productive_hours = production_data.get('productive_hours')
    productive_hours_per_day = production_data.get('productive_hours_per_day')
    
    panel_params = production_data.get('panel_params')
    df_with_production = production_data.get('df_with_production')
    
    # Print everything
    print("\nPRODUCTION SUMMARY")
    print("-" * 40)
    print(f"February total: {february_total_kwh:.2f} kWh")
    print(f"Days in February: {february_days}")
    print(f"Average daily: {avg_daily_kwh:.2f} kWh/day")
    print(f"Maximum daily: {max_daily_kwh:.2f} kWh/day")
    print(f"Minimum daily: {min_daily_kwh:.2f} kWh/day")
    
    print(f"\n📅 BEST AND WORST DAYS")
    print("-" * 40)
    print(f"Best day: {best_day} with {best_day_value:.2f} kWh")
    print(f"Worst day: {worst_day} with {worst_day_value:.2f} kWh")
    
    if worst_day_value > 0:
        ratio = best_day_value / worst_day_value
        print(f"Ratio best/worst: {ratio:.1f}x")
    else:
        print(f"Ratio best/worst: Undefined (zero production day)")
    
    print(f"\n⏰ HOURLY PRODUCTION")
    print("-" * 40)
    print(f"Peak production hour: {peak_hour}:00 ({peak_hour_value:.3f} kWh)")
    print(f"Total productive hours: {productive_hours}")
    print(f"Productive hours per day: {productive_hours_per_day:.1f} hours")
    
    print(f"\n🔧 PANEL CONFIGURATION")
    print("-" * 40)
    if panel_params:
        for key, value in panel_params.items():
            if key in ['panel_efficiency', 'system_losses', 'temp_loss_coeff', 'derating_factor']:
                print(f"{key}: {value*100:.1f}%")
            elif key == 'panel_power_kw':
                print(f"{key}: {value} kWp")
            else:
                print(f"{key}: {value}")
    
    # Show hourly average if available
    if hourly_avg_production is not None:
        print(f"\nHOURLY AVERAGE PRODUCTION")
        print("-" * 40)
        for hour in sorted(hourly_avg_production.index):
            if hourly_avg_production[hour] > 0:
                print(f"   {hour:02d}:00 - {hourly_avg_production[hour]:.3f} kWh")
    
    # Show first few rows of daily production
    if daily_production_df is not None and isinstance(daily_production_df, pd.DataFrame):
        print(f"\nDAILY PRODUCTION (first 5 days)")
        print("-" * 40)
        print(daily_production_df.head().to_string(index=False))
    
    print("\n" + "=" * 60)
    print("ALL Q2 DATA LOADED SUCCESSFULLY")
    print("=" * 60)
    
else:
    print("Failed to load production_data from Q2")
    print("Please run Q2 notebook first and make sure all variables are saved")
    raise ValueError("Cannot continue without data from Q2")


Libraries imported successfully
LOADING ALL VARIABLES FROM Q2
Loaded production_data dictionary with 17 items

COMPLETE DATA LOADED FROM Q2

PRODUCTION SUMMARY
----------------------------------------
February total: 280.00 kWh
Days in February: 28
Average daily: 10.00 kWh/day
Maximum daily: 0.94 kWh/day
Minimum daily: 0.00 kWh/day

📅 BEST AND WORST DAYS
----------------------------------------
Best day: 2026-02-25 with 0.94 kWh
Worst day: 2026-02-04 with 0.00 kWh
Ratio best/worst: Undefined (zero production day)

⏰ HOURLY PRODUCTION
----------------------------------------
Peak production hour: 12:00 (0.381 kWh)
Total productive hours: 25
Productive hours per day: 0.9 hours

🔧 PANEL CONFIGURATION
----------------------------------------
panel_power_kw: 3.0 kWp
panel_efficiency: 19.0%
system_losses: 14.0%
temp_loss_coeff: 0.4%
optimal_angle: 35
orientation: South
derating_factor: 85.0%

HOURLY AVERAGE PRODUCTION
----------------------------------------
   12:00 - 0.381 kWh

DAILY PROD

In [9]:
#  Define economic parameters
print("=" * 60)
print("ECONOMIC PARAMETERS")
print("=" * 60)

# Get panel power from Q2
panel_power = panel_params.get('panel_power_kw', 3.0)

# Define economic parameters
ECON_PARAMS = {
    # Installation costs
    'installation_cost_per_kw': 1800,       # per kWp installed
    'annual_maintenance': 150,              # per year (cleaning, inspections)
    'inverter_replacement_year': 12,        # Inverter replacement after years
    'inverter_replacement_cost': 1200,      # for new inverter
    
    # Energy costs
    'electricity_rate': 0.30,               # per kWh (what you pay to buy from grid)
    'sell_back_rate': 0.10,                 # per kWh (what you get selling to grid)
    'incentive_rate': 0.12,                 # per kWh (Scambio Sul Posto incentive)
    'annual_rate_increase': 0.03,           # 3% annual increase in electricity prices
    
    # Consumption
    'household_consumption': 2700,          # kWh per year (typical Italian family)
    
    # Analysis period
    'analysis_years': 25,                   # years for ROI calculation
    'discount_rate': 0.02                   # 2% discount rate for NPV
}

installation_cost = panel_power * ECON_PARAMS['installation_cost_per_kw']

print(f"\nPanel power: {panel_power} kWp")
print(f"Installation cost: {installation_cost:,.0f} ( {ECON_PARAMS['installation_cost_per_kw']}/kWp)")
print(f"Annual maintenance: {ECON_PARAMS['annual_maintenance']}/year")
print(f"Electricity rate: {ECON_PARAMS['electricity_rate']}/kWh")
print(f"Sell-back rate: {ECON_PARAMS['sell_back_rate']}/kWh")
print(f"Incentive rate: {ECON_PARAMS['incentive_rate']}/kWh")
print(f"Household consumption: {ECON_PARAMS['household_consumption']} kWh/year")

ECONOMIC PARAMETERS

Panel power: 3.0 kWp
Installation cost: 5,400 ( 1800/kWp)
Annual maintenance: 150/year
Electricity rate: 0.3/kWh
Sell-back rate: 0.1/kWh
Incentive rate: 0.12/kWh
Household consumption: 2700 kWh/year


In [10]:
# Self-consumption analysis based on February
print("\n" + "=" * 60)
print("SELF-CONSUMPTION ANALYSIS - FEBRUARY ONLY")
print("=" * 60)

# Use February total from Q2
print(f"February total production: {february_total_kwh:.2f} kWh")
print(f"Days in February: {february_days}")
print(f"Average daily: {avg_daily_kwh:.2f} kWh/day")

# Monthly consumption estimate (typical Italian family)
monthly_consumption = 225  # kWh per month (2700 kWh/year / 12)

print(f"Estimated monthly consumption: {monthly_consumption} kWh")

# Try different self-consumption percentages
self_consumption_pcts = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
results = []

for sc_pct in self_consumption_pcts:
    # Energy flows for February
    self_consumed = february_total_kwh * sc_pct
    grid_injected = february_total_kwh * (1 - sc_pct)
    
    # Financial benefits for February
    savings = self_consumed * ECON_PARAMS['electricity_rate']
    revenue = grid_injected * ECON_PARAMS['sell_back_rate']
    
    # Incentive (Scambio Sul Posto - on energy injected, up to consumption)
    eligible = min(grid_injected, monthly_consumption)
    incentive = eligible * ECON_PARAMS['incentive_rate']
    
    # Total monthly benefit
    total_monthly_benefit = savings + revenue + incentive
    
    # Extrapolate to annual for payback calculation (simplified)
    estimated_annual_benefit = total_monthly_benefit * 12 - ECON_PARAMS['annual_maintenance']
    
    # Payback calculation
    if estimated_annual_benefit > 0:
        payback_years = installation_cost / estimated_annual_benefit
    else:
        payback_years = float('inf')
    
    results.append({
        'self_consumption_pct': int(sc_pct * 100),
        'self_consumed_kwh': round(self_consumed, 1),
        'grid_injected_kwh': round(grid_injected, 1),
        'savings_euro': round(savings, 2),
        'revenue_euro': round(revenue, 2),
        'incentive_euro': round(incentive, 2),
        'total_feb_benefit_euro': round(total_monthly_benefit, 2),
        'est_annual_benefit_euro': round(estimated_annual_benefit, 2),
        'payback_years': round(payback_years, 1)
    })

# Display results
results_df = pd.DataFrame(results)
print("\nFEBRUARY SCENARIO COMPARISON:")
print(results_df.to_string(index=False))

# Find optimal scenario (lowest payback)
optimal_idx = results_df['payback_years'].idxmin()
optimal_scenario = results_df.loc[optimal_idx]

print("\n" + "=" * 60)
print("OPTIMAL SCENARIO (based on February)")
print("=" * 60)
print(f"Self-consumption: {optimal_scenario['self_consumption_pct']}%")
print(f"February self-consumed: {optimal_scenario['self_consumed_kwh']} kWh")
print(f"February grid injected: {optimal_scenario['grid_injected_kwh']} kWh")
print(f"February savings: {optimal_scenario['savings_euro']}")
print(f"February revenue: {optimal_scenario['revenue_euro']}")
print(f"February incentive: {optimal_scenario['incentive_euro']}")
print(f"Total February benefit: {optimal_scenario['total_feb_benefit_euro']}")
print(f"\nEstimated annual benefit: {optimal_scenario['est_annual_benefit_euro']}/year")
print(f"Payback period: {optimal_scenario['payback_years']} years")


SELF-CONSUMPTION ANALYSIS - FEBRUARY ONLY
February total production: 280.00 kWh
Days in February: 28
Average daily: 10.00 kWh/day
Estimated monthly consumption: 225 kWh

FEBRUARY SCENARIO COMPARISON:
 self_consumption_pct  self_consumed_kwh  grid_injected_kwh  savings_euro  revenue_euro  incentive_euro  total_feb_benefit_euro  est_annual_benefit_euro  payback_years
                   20               56.0              224.0          16.8          22.4           26.88                   66.08                   642.96            8.4
                   30               84.0              196.0          25.2          19.6           23.52                   68.32                   669.84            8.1
                   40              112.0              168.0          33.6          16.8           20.16                   70.56                   696.72            7.8
                   50              140.0              140.0          42.0          14.0           16.80                   72.80

In [14]:
# Long-term cash flow analysis (with seasonal adjustment)

print("\n" + "=" * 60)
print("LONG-TERM CASH FLOW ANALYSIS")
print("=" * 60)

# Seasonal factors for Turin (from Q2 or use defaults)
seasonal_factors = {
    'January': 0.7, 'February': 1.0, 'March': 1.4,
    'April': 1.8, 'May': 2.1, 'June': 2.3,
    'July': 2.4, 'August': 2.2, 'September': 1.9,
    'October': 1.5, 'November': 1.0, 'December': 0.6
}

# Calculate seasonal adjustment factor (average of all months relative to February)
avg_seasonal_factor = sum(seasonal_factors.values()) / len(seasonal_factors)
print(f"February production: {february_total_kwh:.2f} kWh")
print(f"Seasonal adjustment factor: {avg_seasonal_factor:.2f}x")
print(f"(Based on Turin historical patterns)")

# Estimate annual production using seasonal factors
estimated_annual_kwh = february_total_kwh * avg_seasonal_factor
print(f"Estimated annual production: {estimated_annual_kwh:.0f} kWh")

# Calculate estimated annual benefit
optimal_pct = optimal_scenario['self_consumption_pct'] / 100
estimated_self_consumed = estimated_annual_kwh * optimal_pct
estimated_grid_injected = estimated_annual_kwh * (1 - optimal_pct)

estimated_savings = estimated_self_consumed * ECON_PARAMS['electricity_rate']
estimated_revenue = estimated_grid_injected * ECON_PARAMS['sell_back_rate']
estimated_eligible = min(estimated_grid_injected, ECON_PARAMS['household_consumption'])
estimated_incentive = estimated_eligible * ECON_PARAMS['incentive_rate']

estimated_annual_benefit = estimated_savings + estimated_revenue + estimated_incentive - ECON_PARAMS['annual_maintenance']

print(f"Estimated annual benefit: {estimated_annual_benefit:.0f}/year")

# Cash flow analysis
years = list(range(ECON_PARAMS['analysis_years'] + 1))
cash_flow = []
cumulative = []

# Year 0: Initial investment
cash_flow.append(-installation_cost)
cumulative.append(-installation_cost)

for year in range(1, ECON_PARAMS['analysis_years'] + 1):
    # Adjust benefit for electricity price inflation
    yearly_benefit = estimated_annual_benefit * ((1 + ECON_PARAMS['annual_rate_increase']) ** year)
    
    # Inverter replacement cost
    if year == ECON_PARAMS['inverter_replacement_year']:
        yearly_benefit -= ECON_PARAMS['inverter_replacement_cost']
    
    cash_flow.append(yearly_benefit)
    cumulative.append(cumulative[-1] + yearly_benefit)

# Calculate key metrics
payback_year = 999  # Default large value if never pays back

for i, cum in enumerate(cumulative):
    if cum >= 0 and i > 0:  # i>0 to skip year 0
        payback_year = i
        break

roi_20_years = (cumulative[20] / abs(installation_cost)) * 100 if len(cumulative) > 20 else 0
total_profit_20_years = cumulative[20] if len(cumulative) > 20 else cumulative[-1]

print(f"\nInitial investment: {installation_cost:,.0f}")
print(f"Estimated annual benefit (year 1): {estimated_annual_benefit:.0f}")
print(f"Payback period: {payback_year} years")
print(f"ROI after 20 years: {roi_20_years:.0f}%")
print(f"Total profit after 20 years: {total_profit_20_years:,.0f}")

# Show cash flow summary every 5 years
print("\nCash Flow Summary:")
print("Year | Cash Flow | Cumulative")
print("-" * 35)
for year in [0, 5, 10, 15, 20, 25]:
    if year <= ECON_PARAMS['analysis_years']:
        print(f"{year:4d} | {cash_flow[year]:8,.0f} | {cumulative[year]:8,.0f}")

# Store estimated annual values for later cells
annual_estimate_kwh = estimated_annual_kwh
annual_benefit_estimate = estimated_annual_benefit


LONG-TERM CASH FLOW ANALYSIS
February production: 280.00 kWh
Seasonal adjustment factor: 1.57x
(Based on Turin historical patterns)
Estimated annual production: 441 kWh
Estimated annual benefit: -28/year

Initial investment: 5,400
Estimated annual benefit (year 1): -28
Payback period: 999 years
ROI after 20 years: -137%
Total profit after 20 years: -7,383

Cash Flow Summary:
Year | Cash Flow | Cumulative
-----------------------------------
   0 |   -5,400 |   -5,400
   5 |      -33 |   -5,555
  10 |      -38 |   -5,734
  15 |      -44 |   -7,142
  20 |      -51 |   -7,383
  25 |      -59 |   -7,662


In [15]:
# COMPARISON: SOLAR VS GRID ONLY
print("\n" + "=" * 60)
print("COMPARISON: SOLAR VS GRID ONLY")
print("=" * 60)

# Cost without solar (based on estimated annual consumption)
annual_grid_cost = ECON_PARAMS['household_consumption'] * ECON_PARAMS['electricity_rate']

# Calculate 20-year grid cost with inflation
grid_cost_20_years = 0
for year in range(1, 21):
    grid_cost_20_years += annual_grid_cost * ((1 + ECON_PARAMS['annual_rate_increase']) ** year)

# Net benefit of solar (total savings over 20 years)
solar_net_benefit_20_years = total_profit_20_years + installation_cost  # Profit + initial investment = total savings

print(f"Annual electricity cost without solar: {annual_grid_cost:.0f}")
print(f"Total grid cost over 20 years: {grid_cost_20_years:,.0f}")
print(f"Total savings with solar over 20 years: {solar_net_benefit_20_years:,.0f}")
print(f"Net benefit of going solar: {solar_net_benefit_20_years - grid_cost_20_years:,.0f}")

# Break-even electricity rate
break_even_rate = installation_cost / (estimated_annual_kwh * 20) + ECON_PARAMS['sell_back_rate']
print(f"\nBreak-even electricity rate: {break_even_rate:.2f}/kWh")
print(f"(Current rate: {ECON_PARAMS['electricity_rate']}/kWh)")

# Monthly comparison for February
feb_grid_cost = monthly_consumption * ECON_PARAMS['electricity_rate']
feb_solar_benefit = optimal_scenario['total_feb_benefit_euro']

print(f"\nFEBRUARY COMPARISON:")
print(f"   Grid cost: {feb_grid_cost:.2f}")
print(f"   Solar benefit: {feb_solar_benefit:.2f}")
print(f"   February savings: {feb_solar_benefit:.2f}")

# Percentage of coverage
coverage_percentage = (february_total_kwh / monthly_consumption) * 100
print(f"\nFebruary production covers {coverage_percentage:.1f}% of monthly consumption")


COMPARISON: SOLAR VS GRID ONLY
Annual electricity cost without solar: 810
Total grid cost over 20 years: 22,418
Total savings with solar over 20 years: -1,983
Net benefit of going solar: -24,401

Break-even electricity rate: 0.71/kWh
(Current rate: 0.3/kWh)

FEBRUARY COMPARISON:
   Grid cost: 67.50
   Solar benefit: 77.28
   February savings: 77.28

February production covers 124.4% of monthly consumption


In [16]:
# SENSITIVITY ANALYSIS - PANEL SIZE (based on February)

print("\n" + "=" * 60)
print("SENSITIVITY ANALYSIS - PANEL SIZE")
print("=" * 60)

panel_sizes = [2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 6.0]
sensitivity_results = []

# Base production per kW from February
base_production_per_kw = february_total_kwh / panel_power

print(f"Base February production: {february_total_kwh:.2f} kWh with {panel_power} kWp")
print(f"Production per kW: {base_production_per_kw:.2f} kWh/kW in February")

for size in panel_sizes:
    # Scale February production
    size_feb_production = base_production_per_kw * size
    size_cost = size * ECON_PARAMS['installation_cost_per_kw']
    
    # Estimate annual using seasonal factors
    size_annual_production = size_feb_production * avg_seasonal_factor
    
    # Use same self-consumption percentage as optimal scenario
    self_pct = optimal_scenario['self_consumption_pct'] / 100
    
    # Calculate benefits
    self_consumed = size_annual_production * self_pct
    grid_injected = size_annual_production * (1 - self_pct)
    
    savings = self_consumed * ECON_PARAMS['electricity_rate']
    revenue = grid_injected * ECON_PARAMS['sell_back_rate']
    eligible = min(grid_injected, ECON_PARAMS['household_consumption'])
    incentive = eligible * ECON_PARAMS['incentive_rate']
    
    total_annual_benefit = savings + revenue + incentive - ECON_PARAMS['annual_maintenance']
    
    if total_annual_benefit > 0:
        payback = size_cost / total_annual_benefit
    else:
        payback = float('inf')
    
    sensitivity_results.append({
        'panel_size_kw': size,
        'feb_production_kwh': round(size_feb_production, 1),
        'annual_production_kwh': int(size_annual_production),
        'installation_cost_euro': int(size_cost),
        'annual_benefit_euro': int(total_annual_benefit),
        'payback_years': round(payback, 1)
    })

sensitivity_df = pd.DataFrame(sensitivity_results)
print("\nSENSITIVITY RESULTS (based on February + seasonal adjustment):")
print(sensitivity_df.to_string(index=False))

# Find best size (lowest payback)
best_size_idx = sensitivity_df['payback_years'].idxmin()
best_size = sensitivity_df.loc[best_size_idx]

print("\n" + "=" * 60)
print(f"OPTIMAL PANEL SIZE: {best_size['panel_size_kw']} kWp")
print(f"   February production: {best_size['feb_production_kwh']} kWh")
print(f"   Estimated annual: {best_size['annual_production_kwh']} kWh/year")
print(f"   Installation: {best_size['installation_cost_euro']:,}")
print(f"   Annual benefit: {best_size['annual_benefit_euro']}")
print(f"   Payback: {best_size['payback_years']} years")

# Compare with current panel size - FIXED: Add error handling
print("\n" + "-" * 40)
print(f"Current panel size: {panel_power} kWp")

if payback_year is not None and payback_year != 999:
    print(f"Current payback: {payback_year} years")
    print(f"Optimal panel size: {best_size['panel_size_kw']} kWp")
    
    if best_size['payback_years'] < float('inf'):
        improvement = payback_year - best_size['payback_years']
        if improvement > 0:
            print(f"Improvement: {improvement:.1f} years shorter payback")
        else:
            print(f"Current panel size is already optimal")
else:
    print(f"Current payback: Not calculated (may never pay back)")
    print(f"Optimal panel size: {best_size['panel_size_kw']} kWp with {best_size['payback_years']} years payback")


SENSITIVITY ANALYSIS - PANEL SIZE
Base February production: 280.00 kWh with 3.0 kWp
Production per kW: 93.33 kWh/kW in February

SENSITIVITY RESULTS (based on February + seasonal adjustment):
 panel_size_kw  feb_production_kwh  annual_production_kwh  installation_cost_euro  annual_benefit_euro  payback_years
           2.0               186.7                    294                    3600                  -68            inf
           2.5               233.3                    367                    4500                  -48            inf
           3.0               280.0                    441                    5400                  -28            inf
           3.5               326.7                    514                    6300                   -7            inf
           4.0               373.3                    588                    7200                   12          585.9
           4.5               420.0                    661                    8100                  

In [17]:
# ECONOMIC SUMMARY FOR Q4
print("\n" + "=" * 60)
print("CREATING ECONOMIC SUMMARY FOR Q4")
print("=" * 60)

# Get output directory from previous notebooks or set default
try:
    %store -r output_dir
except:
    output_dir = "notebooks_output"

# Ensure payback_year is a number
if payback_year is None or payback_year == 999:
    payback_year_display = "Never pays back"
    payback_year_value = 999
else:
    payback_year_display = f"{payback_year} years"
    payback_year_value = payback_year

economic_data = {
    # Parameters used
    'panel_params': panel_params,
    'econ_params': ECON_PARAMS,
    'panel_power': panel_power,
    'installation_cost': installation_cost,
    
    # February data
    'february_total_kwh': february_total_kwh,
    'february_days': february_days,
    'avg_daily_kwh': avg_daily_kwh,
    'peak_hour': peak_hour,
    'productive_hours': productive_hours,
    
    # Optimal scenario (February)
    'optimal_scenario': optimal_scenario.to_dict() if hasattr(optimal_scenario, 'to_dict') else dict(optimal_scenario),
    'optimal_self_consumption_pct': optimal_scenario['self_consumption_pct'],
    'optimal_feb_benefit': optimal_scenario['total_feb_benefit_euro'],
    
    # Seasonal adjustment
    'seasonal_factors': seasonal_factors,
    'avg_seasonal_factor': avg_seasonal_factor,
    'estimated_annual_kwh': estimated_annual_kwh,
    'estimated_annual_benefit': estimated_annual_benefit,
    
    # Key metrics
    'payback_years': payback_year_value,
    'payback_years_display': payback_year_display,
    'roi_20_years': roi_20_years,
    'total_profit_20_years': total_profit_20_years,
    
    # Grid comparison
    'annual_grid_cost': annual_grid_cost,
    'grid_cost_20_years': grid_cost_20_years,
    'solar_savings_20_years': solar_net_benefit_20_years,
    'net_benefit_vs_grid': solar_net_benefit_20_years - grid_cost_20_years,
    'break_even_rate': break_even_rate,
    'coverage_percentage': coverage_percentage,
    
    # Cash flow
    'cash_flow_years': years,
    'cash_flow_values': cash_flow,
    'cumulative_values': cumulative,
    
    # Sensitivity analysis
    'sensitivity_results': sensitivity_df,
    'optimal_panel_size': best_size['panel_size_kw'],
    'optimal_panel_payback': best_size['payback_years'],
    
    # Dataframes
    'scenario_comparison': results_df
}

print(f"Created economic_data dictionary with {len(economic_data)} items")
print("\nKEY METRICS SUMMARY:")
print("-" * 40)
print(f"February production: {february_total_kwh:.2f} kWh")
print(f"Estimated annual: {estimated_annual_kwh:.0f} kWh/year")
print(f"Optimal self-consumption: {optimal_scenario['self_consumption_pct']}%")
print(f"Installation cost: {installation_cost:,.0f}")
print(f"Payback period: {payback_year_display}")
print(f"ROI after 20 years: {roi_20_years:.0f}%")
print(f"20-year profit: {total_profit_20_years:,.0f}")
print(f"Optimal panel size: {best_size['panel_size_kw']} kWp")


CREATING ECONOMIC SUMMARY FOR Q4
Created economic_data dictionary with 33 items

KEY METRICS SUMMARY:
----------------------------------------
February production: 280.00 kWh
Estimated annual: 441 kWh/year
Optimal self-consumption: 70.0%
Installation cost: 5,400
Payback period: Never pays back
ROI after 20 years: -137%
20-year profit: -7,383
Optimal panel size: 6.0 kWp


In [18]:
# Cell 9: SAVE ALL VARIABLES FOR Q4
print("\n" + "=" * 60)
print("SAVING ALL VARIABLES FOR Q4 (RECOMMENDATIONS)")
print("=" * 60)

# Create output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
print(f"Output directory: {output_dir}")

# Method 1: Save individual key variables
print("\nSaving individual variables with %store...")
key_vars = ['economic_data', 'results_df', 'sensitivity_df', 'payback_year_value', 
            'roi_20_years', 'total_profit_20_years', 'optimal_scenario',
            'estimated_annual_kwh', 'february_total_kwh', 'avg_seasonal_factor']

for var_name in key_vars:
    if var_name in dir():
        try:
            get_ipython().run_line_magic('store', var_name)
            print(f"  {var_name}")
        except:
            print(f"  {var_name} (could not store)")

# Method 2: Save economic dictionary
%store economic_data

# Method 3: Save as pickle backup
backup_file = f"{output_dir}/q3_economic_data.pkl"
with open(backup_file, 'wb') as f:
    pickle.dump(economic_data, f)

print(f"\nBackup saved to: {backup_file}")
file_size = os.path.getsize(backup_file) / 1024
print(f"   File size: {file_size:.1f} KB")

print("\n" + "=" * 60)
print("Q3 ECONOMIC ANALYSIS COMPLETE")
print("=" * 60)

print("\nTO ACCESS VARIABLES IN Q4 (RECOMMENDATIONS):")
print("")
print("   # Method A: Load the economic dictionary")
print("   %store -r economic_data")
print("   payback = economic_data['payback_years_display']")
print("   roi = economic_data['roi_20_years']")
print("   feb_prod = economic_data['february_total_kwh']")
print("")
print("   # Method B: Load individual variables")
print("   %store -r payback_year_value")
print("   %store -r roi_20_years")
print("   %store -r estimated_annual_kwh")
print("")
print("   # Method C: List all stored variables")
print("   %store")


SAVING ALL VARIABLES FOR Q4 (RECOMMENDATIONS)
Output directory: notebooks_output

Saving individual variables with %store...
Stored 'economic_data' (dict)
  economic_data
Stored 'results_df' (DataFrame)
  results_df
Stored 'sensitivity_df' (DataFrame)
  sensitivity_df
Stored 'payback_year_value' (int)
  payback_year_value
Stored 'roi_20_years' (float64)
  roi_20_years
Stored 'total_profit_20_years' (float64)
  total_profit_20_years
Stored 'optimal_scenario' (Series)
  optimal_scenario
Stored 'estimated_annual_kwh' (float)
  estimated_annual_kwh
Stored 'february_total_kwh' (int)
  february_total_kwh
Stored 'avg_seasonal_factor' (float)
  avg_seasonal_factor
Stored 'economic_data' (dict)

Backup saved to: notebooks_output/q3_economic_data.pkl
   File size: 4.9 KB

Q3 ECONOMIC ANALYSIS COMPLETE

TO ACCESS VARIABLES IN Q4 (RECOMMENDATIONS):

   # Method A: Load the economic dictionary
   %store -r economic_data
   payback = economic_data['payback_years_display']
   roi = economic_data['ro